In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords 

from numpy import array 
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

from keras.models import Model
from keras.layers import Dense, Embedding, Input, RepeatVector, Flatten
from keras.layers import LSTM, Bidirectional, Dropout, Permute, Multiply, Lambda
from keras import backend as K

In [3]:
df = pd.read_csv('reviews.csv')
df.isnull().values.any() # checks if data file have any null values
df.head()

,reviews,sentiment
0,"Well, you'd better if you plan on sitting thro...",0
1,Moonwalker is a Fantasy Music film staring Mic...,1
2,I bought this video on a throw-out table at th...,0
3,Since the last horrid Astérix film and the fac...,1
4,"I grew up with the Abbott and Costello movies,...",1


In [4]:
def preprocess(text):
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
  text = re.sub('[\W]+', ' ', text.lower()) +\
      ' '.join(emoticons).replace('-', '')
  return text

In [5]:
X = []
sentences = list(df['reviews'])
for sen in sentences:
    X.append(preprocess(sen))
y = df['sentiment']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(len(X_train[1]))
print(len(X_test[1]))

194
108


In [8]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [10]:
# using GloVe for creating feature matrix
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.300d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [12]:
embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [13]:
len(embedding_matrix)

94562

In [14]:
def attention(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(1, activation='softmax')(a)
    a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
    a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

In [15]:
units=200
vocab_size = embedding_matrix.shape[0]
embedding_size = embedding_matrix.shape[1]
_input = Input(shape=(maxlen, ))

# get the embedding layer
embedded = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_size,
        input_length=maxlen,
        trainable=False,
        mask_zero=False,
        weights=[embedding_matrix]
    )(_input)

#bi-directional lstm layer
bidi_lstm_out = Bidirectional(LSTM(units, return_sequences=True, dropout=0.25,
                           recurrent_dropout=0.25))(embedded)

#adding attention
attention_mul = attention(bidi_lstm_out)
attention_mul = Flatten()(attention_mul)
output = Dense(1, activation='sigmoid')(attention_mul)
model = Model(input=[_input], output=output)

#compiling model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\rawll\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     28368600    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 400)     801600      embedding_1[0][0]                
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 400, 100)     0           bidirectional_1[0][0]            
____________________________________________________________________________________________

In [17]:
model.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_test,y_test))


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 189s 5ms/step - loss: 0.4288 - accuracy: 0.7983 - val_loss: 0.3539 - val_accuracy: 0.8390
Epoch 2/10
40000/40000 [==============================] - 192s 5ms/step - loss: 0.3252 - accuracy: 0.8582 - val_loss: 0.3308 - val_accuracy: 0.8566
Epoch 3/10
40000/40000 [==============================] - 193s 5ms/step - loss: 0.2603 - accuracy: 0.8894 - val_loss: 0.3403 - val_accuracy: 0.8549
Epoch 4/10
40000/40000 [==============================] - 196s 5ms/step - loss: 0.1809 - accuracy: 0.9283 - val_loss: 0.3970 - val_accuracy: 0.8453
Epoch 5/10
40000/40000 [==============================] - 196s 5ms/step - loss: 0.1138 - accuracy: 0.9582 - val_loss: 0.4454 - val_accuracy: 0.8383
Epoch 6/10
40000/40000 [==============================] - 201s 5ms/step - loss: 0.0723 - accuracy: 0.9741 - val_loss: 0.5245 - val_accuracy: 0.8408
Epoch 7/10
40000/40000 [==============================] - 199